<a href="https://colab.research.google.com/github/shadowdesigner/Telco-Customer-Churn/blob/main/Fiddler_Telco_Customer_Churn_monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fiddler Simple Monitoring Quick Start Guide

## Goal

This guide provides a comprehensive tour of Fiddler's monitoring capabilities, demonstrating how to onboard a binary classification model and then layer on advanced features like proactive alerts, custom business metrics, user-defined segments, and multiple baseline strategies. 🗺️

## Deep Dive Into Fiddler's Monitoring and Analytic Capabilities

This notebook serves as a comprehensive introduction to Fiddler's powerful monitoring toolkit, using a customer churn model as the example. After walking through the fundamental steps of onboarding a binary classification model, the guide demonstrates how to build a robust, production-grade monitoring setup. You'll learn how to configure proactive alerts for data quality and performance, translate model metrics into business impact with custom metrics, isolate specific cohorts with segments, and implement both static and rolling baselines for powerful drift analysis. This guide showcases how these features work together to provide a complete view of your model's health.

## About Fiddler

Fiddler is the all-in-one AI Observability and Security platform for responsible AI. Monitoring and analytics capabilities provide a common language, centralized controls, and actionable insights to operationalize production ML models, GenAI, AI agents, and LLM applications with trust. An integral part of the platform, the Fiddler Trust Service provides quality and moderation controls for LLM applications. Powered by cost-effective, task-specific, and scalable Fiddler-developed trust models — including cloud and VPC deployments for secure environments — it delivers the fastest guardrails in the industry. Fortune 500 organizations utilize Fiddler to scale LLM and ML deployments, delivering high-performance AI, reducing costs, and ensuring responsible governance.

### Getting Started

1. Connect to Fiddler
2. Load a Data Sample
3. Define the Model Specifications
4. Set the Model Task
5. Create a Model
6. Set Up Alerts **(Optional)**
7. Create a Custom Metric **(Optional)**
8. Create a Segment **(Optional)**
9. Publish a Pre-production Baseline **(Optional)**
10. Configure a Rolling Baseline **(Optional)**
11. Publish Production Events

# 0. Imports

In [1]:
%pip install -q fiddler-client

import time as time

import pandas as pd
import fiddler as fdl

print(f'Running Fiddler Python client version {fdl.__version__}')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 5.2 MB/s eta 0:00:00
Running Fiddler Python client version 3.10.0


## 1. Connect to Fiddler

Before you can add information about your model with Fiddler, you'll need to connect using the Fiddler Python client.


---


**We need a couple pieces of information to get started.**
1. The URL you're using to connect to Fiddler
2. Your authorization token

Your authorization token can be found by navigating to the **Credentials** tab on the **Settings** page of your Fiddler environment.

In [5]:
URL = 'https://tech-exercise.cloud.fiddler.ai/'  # Make sure to include the full URL (including https:// e.g. 'https://your_company_name.fiddler.ai').
TOKEN = 'tbZHTVFYS63Ilw0H0u64KNs0k3cQ9bmPVGaovfxcYg4'

Constants for this example notebook, change as needed to create your own versions

In [10]:
PROJECT_NAME = 'noah_tech_exercise'  # If the project already exists, the notebook will create the model under the existing project.
MODEL_NAME = 'telco_customer_churn_monitoring'

STATIC_BASELINE_NAME = 'baseline_dataset'
ROLLING_BASELINE_NAME = 'rolling_baseline_1week'

# Sample data hosted on GitHub
PATH_TO_SAMPLE_CSV = 'https://raw.githubusercontent.com/shadowdesigner/Telco-Customer-Churn/refs/heads/main/WA_Fn-UseC_-Telco-Customer-Churn.csv'
#PATH_TO_EVENTS_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/churn_production_data.csv'

Now just run the following to connect to your Fiddler environment.

In [11]:
fdl.init(url=URL, token=TOKEN)

#### 1.a Create New or Load Existing Project

Once you connect, you can create a new project by specifying a unique project name in the fld.Project constructor and calling the `create()` method. If the project already exists, it will load it for use.

In [13]:
project = fdl.Project.get_or_create(name=PROJECT_NAME)

print(f'Using project with id = {project.id} and name = {project.name}')

Using project with id = 18930082-8f31-4537-bd7e-ba23a7112d84 and name = noah_tech_exercise


You should now be able to see the newly created project in the Fiddler UI.

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/simple_monitoring_1.png" />
        </td>
    </tr>
</table>

## 2. Load a Data Sample

In this example, we'll be considering the case where we're a bank and we have **a model that predicts churn for our customers**.
  
In order to get insights into the model's performance, **Fiddler needs a small sample of data** to learn the schema of incoming data.

In [14]:
sample_data_df = pd.read_csv(PATH_TO_SAMPLE_CSV)
column_list  = sample_data_df.columns
sample_data_df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


## 3. Define the Model Specifications

In order to create a model in Fiddler, create a ModelSpec object with information about what each column of your data sample should used for.

Fiddler supports four column types:
1. **Inputs**
2. **Outputs** (Model predictions)
3. **Target** (Ground truth values)
4. **Metadata**

In [17]:
input_columns = list(
    #column_list.drop(['predicted_churn', 'churn', 'customer_id', 'timestamp'])
    column_list.drop(['MonthlyCharges', 'Churn', 'customerID', 'tenure'])
)

In [20]:
model_spec = fdl.ModelSpec(
    inputs=input_columns,
    outputs=['MonthlyCharges'],
    targets=['Churn'],  # Note: only a single Target column is allowed, use metadata columns and custom metrics for additional targets
    metadata=['customerID', 'tenure'],
)

If you have columns in your ModelSpec which denote **prediction IDs or timestamps**, then Fiddler can use these to power its analytics accordingly.

Let's call them out here and use them when configuring the Model in step 5.

In [22]:
id_column = 'customerID'
timestamp_column = 'tenure'

## 4. Set the Model Task

Fiddler supports a variety of model tasks. In this case, we're adding a binary classification model.

For this, we'll create a ModelTask object and an additional ModelTaskParams object to specify the ordering of our positive and negative labels.

*For a detailed breakdown of all supported model tasks, click [here](https://docs.fiddler.ai/technical-reference/python-client-guides/explainability/model-task-examples).*

In [25]:
model_task = fdl.ModelTask.BINARY_CLASSIFICATION

task_params = fdl.ModelTaskParams(target_class_order=['No', 'Yes'])

## 5. Create a Model

Create a Model object and publish it to Fiddler, passing in
1. Your data sample
2. The ModelSpec object
3. The ModelTask and ModelTaskParams objects
4. The ID and timestamp columns

In [26]:
model = fdl.Model.from_data(
    name=MODEL_NAME,
    project_id=project.id,
    source=sample_data_df,
    spec=model_spec,
    task=model_task,
    task_params=task_params,
    event_id_col=id_column,
    event_ts_col=timestamp_column,
)

model.create()
print(f'New model created with id = {model.id} and name = {model.name}')

New model created with id = 5753f20c-1b7d-4a3f-ab94-aeaf7b537aea and name = telco_customer_churn_monitoring


On the project page, you should now be able to see the newly onboarded model with its model schema.

<table>
    <tr>
        <td>
            <img src="https://github.com/fiddler-labs/fiddler-examples/blob/main/quickstart/images/simple_monitoring_3.png?raw=true" />
        </td>
    </tr>
</table>

<table>
    <tr>
        <td>
            <img src="https://github.com/fiddler-labs/fiddler-examples/blob/main/quickstart/images/simple_monitoring_4.png?raw=true" />
        </td>
    </tr>
</table>

## 6. Set Up Alerts (Optional)

Fiddler allows creating alerting rules when your data or model predictions deviate from expected behavior.

The alert rules can compare metrics to **absolute** or **relative** values.

Please refer to [our documentation](https://docs.fiddler.ai/technical-reference/python-client-guides/alerts-with-fiddler-client) for more information on Alert Rules.

---
  
Let's set up some alert rules.

The following API call sets up a Data Integrity type rule which triggers an email notification when published events have 2 or more range violations in any 1 day bin for the `numofproducts` column.

In [27]:
alert_rule_1 = fdl.AlertRule(
    name='Telco Churn Range Violation Alert',
    model_id=model.id,
    metric_id='range_violation_count',
    bin_size=fdl.BinSize.DAY,
    compare_to=fdl.CompareTo.RAW_VALUE,
    priority=fdl.Priority.HIGH,
    warning_threshold=500,
    critical_threshold=1000,
    condition=fdl.AlertCondition.GREATER,
    columns=['TotalCharges'],
)

alert_rule_1.create()
print(
    f'New alert rule created with id = {alert_rule_1.id} and name = {alert_rule_1.name}'
)

# Set notification configuration for the alert rule, a single email address for this simple example
alert_rule_1.set_notification_config(emails=['name@google.com'])

New alert rule created with id = d496dbe0-398c-4aa2-809e-6192d114b962 and name = Telco Churn Range Violation Alert


NotificationConfig(emails=['name@google.com'], pagerduty_services=[], pagerduty_severity='', webhooks=[])

Let's add a second alert rule.

This one sets up a Performance type rule which triggers an email notification when precision metric is 5% higher than that from 1 hr bin one day ago.

In [28]:
alert_rule_2 = fdl.AlertRule(
    name='Telco Churn Performance Alert',
    model_id=model.id,
    metric_id='precision',
    bin_size=fdl.BinSize.HOUR,
    compare_to=fdl.CompareTo.TIME_PERIOD,
    compare_bin_delta=24,  # Multiple of the bin size
    condition=fdl.AlertCondition.GREATER,
    warning_threshold=0.05,
    critical_threshold=0.1,
    priority=fdl.Priority.HIGH,
)

alert_rule_2.create()
print(
    f'New alert rule created with id = {alert_rule_2.id} and name = {alert_rule_2.name}'
)

# Set notification configuration for the alert rule, a single email address for this simple example
alert_rule_2.set_notification_config(emails=['name@google.com'])

New alert rule created with id = 510e27b9-d6a5-4103-b29b-a4ae891b0a06 and name = Telco Churn Performance Alert


NotificationConfig(emails=['name@google.com'], pagerduty_services=[], pagerduty_severity='', webhooks=[])

## 7. Create a Custom Metric (Optional)

Fiddler's [Custom Metrics](https://docs.fiddler.ai/technical-reference/api-methods-30#custom-metrics) feature enables user-defined formulas for custom metrics.  Custom metrics will be tracked over time and can be used in Charts and Alerts just like the many out of the box metrics provided by Fiddler.  Custom metrics can also be managed in the Fiddler UI.

Please refer to [our documentation](https://docs.fiddler.ai/product-guide/monitoring-platform/custom-metrics) for more information on Custom Metrics.

---
  
Let's create an example custom metric.

In [29]:
custom_metric = fdl.CustomMetric(
    name='Lost Revenue',
    model_id=model.id,
    description='A metric to track revenue lost for each false positive prediction.',
    definition="""sum(if(fp(),1,0) * -100)""",  # This is an excel like formula which adds -$100 for each false positive predicted by the model
)

custom_metric.create()
print(
    f'New custom metric created with id = {custom_metric.id} and name = {custom_metric.name}'
)

New custom metric created with id = 73c9fd99-3518-422b-8532-03ed9c3064cc and name = Lost Revenue


## 8. Create a Segment (Optional)
Fiddler's [Segment API](https://docs.fiddler.ai/technical-reference/api-methods-30#segments) enables defining named cohorts/sub-segments in your production data. These segments can be tracked over time, added to charts, and alerted upon. Segments can also be managed in the Fiddler UI.

Please refer to our [documentation](https://docs.fiddler.ai/product-guide/monitoring-platform/segments) for more information on the creation and management of segments.

Let's create a segment to track customers from Hawaii for a specific age range.

In [30]:
segment = fdl.Segment(
    name='Senior Customers with tenure between 30 and 60',
    model_id=model.id,
    description='Senior Customers with tenure between 30 and 60',
    definition="(tenure<60 and tenure>30) and InternetService=='DSL'",
)

segment.create()
print(f'New segment created with id = {segment.id} and name = {segment.name}')

New segment created with id = 0b0392d0-9540-4e7a-a78e-0b05daecf2c0 and name = Senior Customers with tenure between 30 and 60


## 9. Publish a Static Baseline (Optional)

Since Fiddler already knows how to process data for your model, we can now add a **baseline dataset**.

You can think of this as a static dataset which represents **"golden data,"** or the kind of data your model expects to receive.

Then, once we start sending production data to Fiddler, you'll be able to see **drift scores** telling you whenever it starts to diverge from this static baseline.

***

Let's publish our **original data sample** as a pre-production dataset and then explicitly create a baseline from it.

**Note:** As of recent updates, baseline creation is now a separate, explicit step after uploading pre-production data. This gives you more control over when and how baselines are created.


*For more information on how to design your baseline dataset, [click here](https://docs.fiddler.ai/technical-reference/python-client-guides/publishing-production-data/creating-a-baseline-dataset).*

In [31]:
# Upload the pre-production data to make it available as a baseline
baseline_publish_job = model.publish(
    source=sample_data_df,
    environment=fdl.EnvType.PRE_PRODUCTION,
    dataset_name=STATIC_BASELINE_NAME,
)
print(
    f'Initiated pre-production environment data upload with Job ID = {baseline_publish_job.id}'
)

# Uncomment the line below to wait for the job to finish, otherwise it will run in the background.
# You can check the status on the Jobs page in the Fiddler UI or use the job ID to query the job status via the API.
# baseline_publish_job.wait()

Initiated pre-production environment data upload with Job ID = c4b8afa0-2b2d-4916-9f95-ef742adcabb9


Now we need to **explicitly create a baseline** from the uploaded data. This step is required as automatic baseline creation has been removed from the pre-production data upload process.

## 10. Configure a Rolling Baseline (Optional)

Fiddler also allows you to configure a baseline based on **past production data.**

This means instead of looking at a static slice of data, it will look into past production events and use what it finds for drift calculation.

Please refer to [our documentation](https://docs.fiddler.ai/technical-reference/python-client-guides/publishing-production-data/creating-a-baseline-dataset) for more information on Baselines.

---
  
Let's set up a rolling baseline that will allow us to calculate drift relative to production data from 1 week back.

In [32]:
rolling_baseline = fdl.Baseline(
    model_id=model.id,
    name=ROLLING_BASELINE_NAME,
    type_=fdl.BaselineType.ROLLING,
    environment=fdl.EnvType.PRODUCTION,
    window_bin_size=fdl.WindowBinSize.DAY,  # Size of the sliding window
    offset_delta=7,  # How far back to set our window (multiple of window_bin_size)
)

rolling_baseline.create()
print(
    f'New rolling baseline created with id = {rolling_baseline.id} and name = {rolling_baseline.name}'
)

New rolling baseline created with id = 2674bf1f-9fc2-4f73-b2a1-5889eab146c7 and name = rolling_baseline_1week


## 11. Publish Production Events

Finally, let's send in some production data!


Fiddler will **monitor this data and compare it to your baseline to generate powerful insights into how your model is behaving**.


---


Each record sent to Fiddler is called **an event**.
  
Let's load some sample events from a CSV file.

In [34]:
production_data_df = pd.read_csv(PATH_TO_EVENTS_CSV)

# Shift the timestamps of the production events to be as recent as today
production_data_df['tenure'] = production_data_df['tenure'] + (
    int(time.time() * 1000) - production_data_df['tenure'].max()
)
production_data_df

NameError: name 'PATH_TO_EVENTS_CSV' is not defined

In [ ]:
production_publish_job = model.publish(production_data_df)

print(
    f'Initiated production environment data upload with Job ID = {production_publish_job.id}'
)

# Uncomment the line below to wait for the job to finish, otherwise it will run in the background.
# You can check the status on the Jobs page in the Fiddler UI or use the job ID to query the job status via the API.
# production_publish_job.wait()

# Get Insights
  
Return to your Fiddler environment to get enhanced observability into your model's performance.

<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/simple_monitoring_5.png" />
        </td>
    </tr>
</table>

**What's Next?**

Try the [LLM Monitoring - Quick Start Notebook](https://docs.fiddler.ai/tutorials-and-quick-starts/llm-and-genai/simple-llm-monitoring)

---


**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.